# Assignment 1C - Question 2
## Semantic Person Search

In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import keras
from keras import layers


import os
import datetime
import numpy

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorboard import notebook
from tensorflow.keras.preprocessing.image import Iterator

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import pydot
import IPython
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

import cv2

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
train = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Train.csv')
test = pd.read_csv('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Test.csv')

In [3]:
train_img = []
gnd_img = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Train_Data/Originals/*.png')
for myfile in files:
    image = cv2.imread(myfile, 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gnd_img.append(myfile[58:])
    train_img.append(image)

train_img = np.array(train_img)
gnd_img = np.array(gnd_img)

print('Training shape: ', train_img.shape)
print('Training gnd: ', gnd_img[1:10])

Training shape:  (520,)
Training gnd:  ['A_0001_01_083.png' 'A_0001_03_126.png' 'A_0001_05_051.png'
 'A_0002_01_021.png' 'A_0002_04_011.png' 'A_0002_08_051.png'
 'A_0003_03_131.png' 'A_0003_04_026.png' 'A_0003_08_091.png']
<ipython-input-3-f4c1eb5ab4d3>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_img = np.array(train_img)


In [4]:
test_originals = []
gnd_test = []
files = glob.glob('CAB420_Assessment_1C_Data/Data/Q2/Q2/Test_Data/Originals/*.png')
for myfile in files:
    image = cv2.imread(myfile)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gnd_test.append(myfile[57:])
    test_originals.append(image)

test_originals = np.array(test_originals)
gnd_test = np.array(gnd_test)
print('Testing shape: ', test_originals.shape)
print('Testing gnd: ', gnd_test[1:10])

<ipython-input-4-1cf7ccf836f0>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  test_originals = np.array(test_originals)
Testing shape:  (196,)
Testing gnd:  ['cam3-2_frame_2500_pos_419_514_86_253.png'
 'cam3-2_frame_4000_pos_457_520_110_276.png'
 'cam3-2_frame_7750_pos_348_432_159_358.png'
 'cam3_frame_17250_pos_295_380_292_496.png'
 'cam3_frame_19000_pos_400_505_215_423.png'
 'cam3_frame_23750_pos_416_502_134_319.png'
 'cam3_frame_24250_pos_361_469_233_465.png'
 'cam3_frame_25750_pos_355_455_158_342.png'
 'cam3_frame_33000_pos_316_412_160_354.png']


In [5]:
train = train.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])
train_gender = train.iloc[:1]
train_tortyp = train.iloc[:2]
train_torcol = train.iloc[:3]
train_legtyp = train.iloc[:4]
train_legcol = train.iloc[:5]
train_luggage = train.iloc[:6] 



In [6]:
test = test.drop(columns=['torcol2','torcol3','tortex','torcol3','legcol2','legcol3','legtex','pose'])
print(test)

                                              filename  gender  tortyp  \
0    ip54-subjects_frame_14800_pos_478_703_223_649.png       0       1   
1    ip53-subjects_frame_52600_pos_516_701_242_627.png       0       1   
2    ip70-subjects_frame_164400_pos_193_294_127_384...       0       1   
3     ip53-subjects_frame_7600_pos_638_783_169_453.png       1       0   
4    ip53-subjects_frame_61600_pos_520_700_283_681.png       1       1   
..                                                 ...     ...     ...   
191            cam8_frame_58750_pos_269_365_92_303.png       0       0   
192  ip70-subjects_frame_43800_pos_779_948_291_601.png       1       1   
193  ip54-subjects_frame_183400_pos_308_506_196_521...       0       0   
194   ip70-subjects_frame_82200_pos_825_927_75_290.png       0       1   
195  ip53-subjects_frame_50800_pos_471_638_176_481.png       0       1   

     torcol  legtyp  legcol  luggage  
0         8       1       1        0  
1         1       0       4      

In [7]:
#fig = plt.figure(figsize=[20, 20])
#for i in range(100):
#    ax = fig.add_subplot(10, 10, i + 1)
#    ax.imshow(test_originals[i])

In [8]:
inputs = keras.Input(shape=(144, 226, 3, ), name='img')
x = layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu')(inputs)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2, 2))(x)
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = layers.Flatten()(x)
x1 = layers.Dense(64, activation='relu')(x)
gender = layers.Dense(3, name='gender_out')(x1)

x2 = layers.Dense(64, activation='relu')(x)
tortyp = layers.Dense(3, name='tortyp_out')(x2)

x3 = layers.Dense(64, activation='relu')(x)
torcol = layers.Dense(11, name='torcol_out')(x3)

x4 = layers.Dense(64, activation='relu')(x)
legtyp = layers.Dense(3, name='legtyp_out')(x4)

x5 = layers.Dense(64, activation='relu')(x)
legcol = layers.Dense(11, name='legocol_out')(x5)

x6 = layers.Dense(64, activation='relu')(x)
luggage = layers.Dense(3, name='luggage_out')(x6)

model_cnn = keras.Model(inputs=inputs, outputs=[gender,tortyp,torcol,legtyp,legcol,luggage], name='A1CQ2')
model_cnn.summary()

Model: "A1CQ2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 144, 226, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 142, 224, 8)  224         img[0][0]                        
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 71, 112, 8)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 69, 110, 16)  1168        max_pooling2d[0][0]              
______________________________________________________________________________________________

In [9]:
model_cnn.compile(loss=['mean_squared_error', keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
              optimizer=keras.optimizers.RMSprop())

In [10]:
history = model_cnn.fit(train_img, [train_gender, y_train],
                    batch_size=64,
                    epochs=20,
                    validation_data=(x_test_rot, [y_test_rot, y_test]))

SyntaxError: invalid syntax (<ipython-input-10-e0a1468d32bf>, line 1)